# Factor IC Analysis

This notebook evaluates **factor predictive power** using **Information Coefficient (IC)** and **Information Ratio (IR)**.
We compute daily IC between factor values and forward returns, visualize results, and assess factor quality.

---


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import spearmanr


## 1. Generate Synthetic Factor & Returns Data

In [ ]:

np.random.seed(42)
n_assets, n_days = 100, 250

# Simulated factor values (e.g., value, momentum, quality scores)
factors = pd.DataFrame(
    np.random.randn(n_days, n_assets),
    index=pd.date_range("2023-01-01", periods=n_days, freq="B")
)

# Simulated forward returns with some signal correlation
noise = np.random.randn(n_days, n_assets)
forward_returns = 0.05 * factors.shift(1) + 0.95 * noise

factors.head(), forward_returns.head()


## 2. Compute Daily Information Coefficient (Spearman Rank)

In [ ]:

def compute_daily_ic(factors: pd.DataFrame, returns: pd.DataFrame):
    ics = []
    for dt in factors.index:
        x = factors.loc[dt].values
        y = returns.loc[dt].values
        if np.isnan(x).any() or np.isnan(y).any():
            continue
        ic, _ = spearmanr(x, y)
        ics.append((dt, ic))
    return pd.Series(dict(ics))

ics = compute_daily_ic(factors, forward_returns)
ics.head()


## 3. Plot IC Time Series

In [ ]:

plt.figure(figsize=(12,4))
ics.plot()
plt.axhline(0, color="black", lw=1)
plt.title("Daily Information Coefficient (IC)")
plt.ylabel("Spearman Rank IC")
plt.show()


## 4. Summary Statistics

In [ ]:

mean_ic = ics.mean()
std_ic = ics.std()
ir = mean_ic / std_ic # type: ignore

print(f"Mean IC: {mean_ic:.4f}")
print(f"Std IC: {std_ic:.4f}")
print(f"Information Ratio (IR): {ir:.4f}")


## 5. Distribution of ICs

In [ ]:

plt.figure(figsize=(8,4))
ics.hist(bins=30, edgecolor="black")
plt.title("Distribution of Daily ICs")
plt.xlabel("IC value")
plt.ylabel("Frequency")
plt.show()
